# Домашнее задание №1

На семинаре мы проходили метрики, необходимо реализовать некоторые из них

In [1]:
import numpy as np
import pandas as pd

In [2]:
data = pd.DataFrame({'user_id': ['u1','u2','u3'],
                    'recommended_list': [
                        [143, 156, 1134, 991, 27],
                        [1543, 3345, 533, 11, 43],
                        [156, 3345, 10, 15, 1134]
                    ],
                    'bought_list': [
                        [156, 27],
                        [11, 43],
                        [1]
                    ]})
data

,user_id,recommended_list,bought_list
0,u1,"[143, 156, 1134, 991, 27]","[156, 27]"
1,u2,"[1543, 3345, 533, 11, 43]","[11, 43]"
2,u3,"[156, 3345, 10, 15, 1134]",[1]


In [3]:
def hit_rate(recommended_list, bought_list): 
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    flags = np.isin(bought_list, recommended_list)
    hit_rate = int(flags.sum() > 0)    
    
    return hit_rate


def hit_rate_at_k(recommended_list, bought_list, k=5):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    flags = np.isin(recommended_list[:k], bought_list)
    hit_rate = int(flags.sum() > 0)    
    
    return hit_rate

In [4]:
data.apply(lambda x: hit_rate_at_k(x[1], x[2], 5), 1).mean()

0.6666666666666666

In [5]:
def precision(recommended_list, bought_list):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    flags = np.isin(bought_list, recommended_list)

    precision = flags.sum() / len(recommended_list)
    
    return precision


def precision_at_k(recommended_list, bought_list, k=5):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    bought_list = bought_list  # Тут нет [:k] !!
    recommended_list = recommended_list[:k]
    
    flags = np.isin(bought_list, recommended_list)
    
    precision = flags.sum() / len(recommended_list)
    
    return precision


def money_precision_at_k(recommended_list, bought_list, prices_recommended, k=5):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    prices_recommended = np.array(prices_recommended)
    
    recommended_list = recommended_list[:k]
    prices_recommended = prices_recommended[:k]
    
    flags = np.isin(bought_list, recommended_list)
    
    precision = (flags*prices_recommended).sum() / prices_recommended.sum()
    
    return precision

In [6]:
data.apply(lambda x: precision(x[1], x[2]), 1).mean()

0.26666666666666666

## Задание 1. Реализовать метрики Recall@k и  Money Recall@k

*Recall* - доля рекомендованных товаров среди релевантных = Какой % купленных товаров был среди рекомендованных

$$\Large Recall@K(i) = \frac {\sum_{j=1}^{K}\mathbb{1}_{r_{ij}}}{|Rel_i|}$$

$\Large |Rel_i|$ -- количество релевантных товаров для пользователя $i$

$$\Large MoneyRecall@K(i) = \frac {\sum_{j=1}^{K}\mathbb{1}_{r_{ij}}\cdot Price(j)}{\sum_{s\in Rel_i}Price(s)}$$


In [7]:
def recall(recommended_list, bought_list): 
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    flags = np.isin(bought_list, recommended_list)
    
    recall = flags.sum() / len(bought_list)
    
    return recall


def recall_at_k(recommended_list, bought_list, k=5):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    bought_list = bought_list
    recommended_list = recommended_list[:k]
    
    flags = np.isin(bought_list, recommended_list)
    
    recall = flags.sum() / len(bought_list)
    
    return recall


def money_recall_at_k(recommended_list, bought_list, prices_recommended, prices_bought, k=5):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    prices_recommended = np.array(prices_recommended)
    prices_bought = np.array(prices_bought)
    
    recommended_list = recommended_list[:k]
    prices_recommended = prices_recommended[:k]
    
    flags = np.isin(bought_list, recommended_list)
    
    recall = (flags*prices_recommended).sum() / prices_bought.sum()
    
    return recall

## Задание 2. Реализовать метрику MRR@k

Mean Reciprocal Rank

- Считаем для первых k рекоммендаций
- Найти ранк первого релевантного предсказания $\Large rank_j$
- Посчитать reciprocal rank = $\Large\frac{1}{rank_j}$

$$\Large  MMR(i)@k=\frac {1}{\min\limits_{j\in Rel(i)} rank_j}$$

In [8]:
def reciprocal_rank_at_k(recommended_list, bought_list, k):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    recommended_list = recommended_list[:k]
    rank=[0]
    flag = False
    for item_rec in recommended_list:
        for i, item_bought in enumerate(bought_list):
            if item_rec == item_bought:
                rank.append(1 / (i+1))
    rank = max(rank)
        
    return rank

In [9]:
def mrr_at_k(data, k):
    return data.apply(lambda x: reciprocal_rank_at_k(x[1], x[2], k), 1).mean()  

In [10]:
mrr_at_k(data, 4)

0.6666666666666666

## Задание 3*. Реализовать метрику nDCG@k
Normalized discounted cumulative gain. Эту метрику реализовать будет немного сложнее.

$$\Large DCG@K(i) = \sum_{j=1}^{K}\frac{\mathbb{1}_{r_{ij}}}{\log_2 (j+1)}$$


$\Large \mathbb{1}_{r_{ij}}$ -- индикаторная функция показывает что пользователь $i$ провзаимодействовал с продуктом $j$

Для подсчета $nDCG$ нам необходимо найти максимально возможный $DCG$ для пользователя $i$  и рекомендаций длины $K$.
Максимальный $DCG$ достигается когда мы порекомендовали максимально возможное количество релевантных продуктов и все они в начале списка рекомендаций.

$$\Large IDCG@K(i) = max(DCG@K(i)) = \sum_{j=1}^{K}\frac{\mathbb{1}_{j\le|Rel_i|}}{\log_2 (j+1)}$$

$$\Large nDCG@K(i) = \frac {DCG@K(i)}{IDCG@K(i)}$$

$\Large |Rel_i|$ -- количество релевантных продуктов для пользователя $i$



In [11]:
def ndcg_at_k(recommended_list, bought_list, k):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    recommended_list = recommended_list[:k]
    
    flags = np.isin(bought_list, recommended_list)
    
    dcg = 0
    idcg = 0
    if len(flags) < k:
        k = len(flags)
    for i in range(k):
        dcg_i = (flags[i]*1)/np.log2(i+2)
        idcg_i = 1/np.log2(i+2)
        dcg += dcg_i
        idcg += idcg_i
        
    ndcg_k = dcg / idcg
    
    return ndcg_k

In [12]:
recommended_list = [143, 156, 1134, 991, 27, 1543, 3345, 533, 11, 43] #id товаров
bought_list = [156, 32, 143, 991]

In [13]:
ndcg_at_k(recommended_list, bought_list, 3)

0.7039180890341347

In [16]:
data.apply(lambda x: ndcg_at_k(x[1], x[2], 5), 1).mean()  

0.6666666666666666